<a href="https://colab.research.google.com/github/Tom-Jung/Tom-Jung/blob/main/API_DART_F_%EC%97%91%EC%8A%A4%EC%97%A0%EC%97%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install dart-fss

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 MB 11.1 MB/s eta 0:00:00
  Attempting uninstall: termcolor
    Found existing installation: termcolor 2.5.0
    Uninstalling termcolor-2.5.0:
      Successfully uninstalled termcolor-2.5.0


In [4]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 77.6 MB/s eta 0:00:00


In [6]:
import pdfplumber
import pandas as pd
import re

def extract_financial_statements(pdf_path, keywords=["재무상태표", "손익계산서", "현금흐름표"]):
    data_frames = {}  # 결과 저장용 딕셔너리

    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            text = page.extract_text()

            if text:
                # 재무제표 포함된 페이지 찾기
                for keyword in keywords:
                    if keyword in text:
                        print(f"✅ {keyword} 발견 (페이지 {page_num + 1})")

                        # 테이블 데이터 추출
                        tables = page.extract_tables()
                        if tables:
                            for table in tables:
                                df = pd.DataFrame(table)

                                # 데이터 정제: 첫 번째 행을 컬럼명으로 설정
                                df.columns = df.iloc[0]
                                df = df[1:].reset_index(drop=True)

                                data_frames[keyword] = df
                        break

    return data_frames

# 실행 예시
pdf_path = "audit_report.pdf"  # 분석할 감사보고서 파일
financial_data = extract_financial_statements(pdf_path)

# 결과 저장
for statement, df in financial_data.items():
    df.to_csv(f"{statement}.csv", index=False, encoding="utf-8-sig")
    print(f"📁 {statement}.csv 저장 완료")


✅ 재무상태표 발견 (페이지 3)
✅ 손익계산서 발견 (페이지 21)
✅ 재무상태표 발견 (페이지 22)
✅ 손익계산서 발견 (페이지 24)
✅ 손익계산서 발견 (페이지 26)
✅ 재무상태표 발견 (페이지 27)
✅ 손익계산서 발견 (페이지 28)
✅ 재무상태표 발견 (페이지 29)
✅ 재무상태표 발견 (페이지 35)
✅ 재무상태표 발견 (페이지 36)
✅ 손익계산서 발견 (페이지 37)
✅ 재무상태표 발견 (페이지 45)
✅ 재무상태표 발견 (페이지 55)
✅ 손익계산서 발견 (페이지 75)
✅ 현금흐름표 발견 (페이지 83)
✅ 재무상태표 발견 (페이지 88)
✅ 손익계산서 발견 (페이지 116)
📁 손익계산서.csv 저장 완료
📁 재무상태표.csv 저장 완료
📁 현금흐름표.csv 저장 완료


#
:------------------------------------------------------------------------------L2------------------------------------

In [10]:
import requests

# API 키 설정 (DART에서 발급받은 API 키 입력)
API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"

# 삼성전자 종목코드
COMPANY_CODE = "00126380"

# DART API URL (재무제표 XML 다운로드)
URL = f"https://opendart.fss.or.kr/api/fnlttSinglAcnt.xml?crtfc_key={API_KEY}&corp_code={COMPANY_CODE}&bsns_year=2023&reprt_code=11011"

# API 요청 및 XML 저장
response = requests.get(URL)

if response.status_code == 200:
    with open("samsung_financial.xml", "wb") as file:
        file.write(response.content)
    print("✅ XML 다운로드 완료: samsung_financial.xml")
else:
    print("❌ API 요청 실패:", response.status_code)


✅ XML 다운로드 완료: samsung_financial.xml


#
:------------------------------------------------------------------------------L3------------------------------------




In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd

def extract_financial_from_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    financial_data = []

    for item in root.findall(".//list"):
        account_name = item.find("account_nm").text  # 계정 항목 (예: 자산총계)
        current_amount = item.find("thstrm_amount").text  # 올해 금액
        previous_amount = item.find("frmtrm_amount").text  # 전년도 금액

        financial_data.append([account_name, current_amount, previous_amount])

    # DataFrame 생성
    df = pd.DataFrame(financial_data, columns=["항목", "당기 금액", "전기 금액"])
    return df

# 실행 예시
xml_path = "samsung_financial.xml"
df = extract_financial_from_xml(xml_path)

# CSV로 저장
df.to_csv("samsung_financial.csv", index=False, encoding="utf-8-sig")
print("📁 재무제표 데이터 저장 완료: samsung_financial.csv")

# 결과 확인
print(df.head())


#
:------------------------------------------------------------------------------L4------------------------------------

In [13]:
import requests
import pandas as pd

# DART API 인증키
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'  # 자신의 인증키로 변경하세요

# 재무제표 API URL
FS_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

# CSV 파일에서 기업리스트 읽기
# 기업 리스트 파일은 '기업리스트.csv'라는 이름으로 가정하며 'code'라는 열을 포함합니다
corporations_df = pd.read_excel("/content/corp_list_test_비상장.xlsx", sheet_name="L1",  dtype={'code': str})

# 데이터 저장을 위한 리스트
financial_data = []

# 각 기업에 대해 재무제표 데이터 요청
for _, row in corporations_df.iterrows():
    corp_code = row['code']  # 'code' 열 사용
    corp_name = row['name'] if 'name' in row else '알 수 없음'  # 'name' 열이 있는 경우 사용

    params = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code,
        'bsns_year': 2023,  # 연도 예시
        'reprt_code': '11011',  # 연결 재무제표
        'fs_div': 'OFS'  # 개별 재무제표 (OFS)
    }

    # DART API 요청
    response = requests.get(FS_URL, params=params)

    if response.status_code == 200:
        data = response.json()

        # 데이터 출력
        print(f"기업 {corp_name}({corp_code})의 응답 데이터: {data}")

        # 재무제표 데이터 처리
        if 'list' in data and data['list']:
            for item in data['list']:
                financial_data.append({
                    '기업 이름': corp_name,
                    '기업 코드': corp_code,
                    '재무제표 항목': item['account_nm'],
                    '2023년': item.get('thstrm_amount', '데이터 없음'),
                    '2022년': item.get('frmtrm_amount', '데이터 없음'),
                    '2021년': item.get('lfy_amount', '데이터 없음')
                })
        else:
            print(f"기업 {corp_name}({corp_code})의 재무제표 데이터가 없습니다.")
    else:
        print(f"기업 {corp_name}({corp_code})에 대한 API 요청 실패: {response.status_code}")

# 결과를 DataFrame으로 변환
df_4 = pd.DataFrame(financial_data)

# 결과를 엑셀 파일로 저장
df_4.to_excel("기업_재무제표_2023년_L4.xlsx", index=False)
print("기업 재무제표 데이터_L4_를 엑셀 파일로 저장했습니다.")

기업 주식회사 엘지씨엔에스(00139834)의 응답 데이터: {'status': '000', 'message': '정상', 'list': [{'rcept_no': '20240401003467', 'reprt_code': '11011', 'bsns_year': '2023', 'corp_code': '00139834', 'sj_div': 'BS', 'sj_nm': '재무상태표', 'account_id': 'ifrs-full_Assets', 'account_nm': '자산총계', 'account_detail': '-', 'thstrm_nm': '제 37 기', 'thstrm_amount': '3689410848000', 'frmtrm_nm': '제 36 기', 'frmtrm_amount': '3590761148000', 'bfefrmtrm_nm': '제 35 기', 'bfefrmtrm_amount': '3072644489000', 'ord': '7', 'currency': 'KRW'}, {'rcept_no': '20240401003467', 'reprt_code': '11011', 'bsns_year': '2023', 'corp_code': '00139834', 'sj_div': 'BS', 'sj_nm': '재무상태표', 'account_id': 'ifrs-full_CurrentAssets', 'account_nm': '유동자산', 'account_detail': '-', 'thstrm_nm': '제 37 기', 'thstrm_amount': '2723025312000', 'frmtrm_nm': '제 36 기', 'frmtrm_amount': '2587250710000', 'bfefrmtrm_nm': '제 35 기', 'bfefrmtrm_amount': '2092251085000', 'ord': '8', 'currency': 'KRW'}, {'rcept_no': '20240401003467', 'reprt_code': '11011', 'bsns_year': '202

#
:------------------------------------------------------------------------------L5------------------------------------

In [20]:
import requests

# DART API 인증키
API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"

# 비상장 외감법인 고유번호 (DART에서 조회 필요)
CORP_CODE = "00141042"

# 조회 기간 설정 (예: 2023년)
START_DATE = "20230101"
END_DATE = "20231231"

# DART API URL (감사보고서 목록 조회)
URL = f"https://opendart.fss.or.kr/api/list.json?crtfc_key={API_KEY}&corp_code={CORP_CODE}&bgn_de={START_DATE}&end_de={END_DATE}&page_no=1&page_count=10"

# API 요청
response = requests.get(URL)
data = response.json()

# 결과 확인
if data["status"] == "000":
    for report in data["list"]:
        if "감사보고서" in report["report_nm"]:  # 감사보고서 필터링
            print(f"📌 감사보고서 발견: {report['report_nm']}, 접수일: {report['rcept_dt']}, 문서번호: {report['rcept_no']}")
else:
    print("❌ API 요청 실패:", data["message"])


In [21]:
import requests

# DART API 인증키
API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"
# 비상장 외감법인 고유번호 (DART에서 조회 필요)
CORP_CODE = "00141042"
# 조회 기간 설정 (예: 2023년)
START_DATE = "20230101"
END_DATE = "20231231"

# 공시 목록 조회 URL
URL = f"https://opendart.fss.or.kr/api/list.json?crtfc_key={API_KEY}&corp_code={CORP_CODE}&bgn_de={START_DATE}&end_de={END_DATE}&page_no=1&page_count=10"

# API 요청
response = requests.get(URL)

if response.status_code == 200:
    data = response.json()
    if data['status'] == '000':
        for report in data['list']:
            print(f"📌 공시명: {report['report_nm']}, 접수일: {report['rcept_dt']}, 문서번호: {report['rcept_no']}")
    else:
        print("❌ 공시 목록 조회 실패:", data['message'])
else:
    print("❌ API 요청 실패:", response.status_code)


📌 공시명: 분기보고서 (2023.09), 접수일: 20231113, 문서번호: 20231113000038
📌 공시명: 주요사항보고서(감자결정), 접수일: 20231103, 문서번호: 20231103000113
📌 공시명: 주요사항보고서(자기주식취득결정), 접수일: 20231103, 문서번호: 20231103000110
📌 공시명: 주요사항보고서(자기주식취득결정), 접수일: 20231103, 문서번호: 20231103000108
📌 공시명: 반기보고서 (2023.06), 접수일: 20230814, 문서번호: 20230814000008
📌 공시명: 분기보고서 (2023.03), 접수일: 20230515, 문서번호: 20230515000017
📌 공시명: 사업보고서 (2022.12), 접수일: 20230330, 문서번호: 20230330001115


In [22]:
import requests

# DART API 인증키
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'

# 비상장 기업 고유번호 (DART에서 조회한 고유번호)
CORP_CODE = '00141042'

# 조회 기간 설정 (예: 2023년)
START_DATE = '20230101'
END_DATE = '20231231'

# DART API URL (공시 목록 조회)
URL = f'https://opendart.fss.or.kr/api/list.json?crtfc_key={API_KEY}&corp_code={CORP_CODE}&bgn_de={START_DATE}&end_de={END_DATE}&page_no=1&page_count=10'

# API 요청
response = requests.get(URL)

# 응답 처리
if response.status_code == 200:
    data = response.json()
    if data['status'] == '000':
        for report in data['list']:
            if '사업보고서' in report['report_nm']:  # 사업보고서 필터링
                print(f"📌 사업보고서 발견: {report['report_nm']}, 접수일: {report['rcept_dt']}, 문서번호: {report['rcept_no']}")
    else:
        print(f"❌ 공시 목록 조회 실패: {data['message']}")
else:
    print(f"❌ API 요청 실패: {response.status_code}")


📌 사업보고서 발견: 사업보고서 (2022.12), 접수일: 20230330, 문서번호: 20230330001115


In [23]:
# 문서번호 (위에서 얻은 문서번호 사용)
DOC_NO = 'DOCUMENT_NUMBER'

# DART API로 XML 다운로드
DOC_URL = f'https://opendart.fss.or.kr/api/document.xml?crtfc_key={API_KEY}&rcept_no={DOC_NO}'

# XML 파일 다운로드
response = requests.get(DOC_URL)

if response.status_code == 200:
    with open('business_report.xml', 'wb') as file:
        file.write(response.content)
    print('✅ 사업보고서 XML 다운로드 완료: business_report.xml')
else:
    print(f'❌ XML 다운로드 실패: {response.status_code}')


✅ 사업보고서 XML 다운로드 완료: business_report.xml


In [27]:
import requests

# DART API 인증키
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'

# 비상장 기업 고유번호 (DART에서 조회한 고유번호)
CORP_CODE = '01507722'

# 조회 기간 설정 (예: 2023년)
START_DATE = '20230101'
END_DATE = '20231231'

# DART API URL (공시 목록 조회)
URL = f'https://opendart.fss.or.kr/api/list.json?crtfc_key={API_KEY}&corp_code={CORP_CODE}&bgn_de={START_DATE}&end_de={END_DATE}&page_no=1&page_count=10'

# API 요청
response = requests.get(URL)

# 응답 처리
if response.status_code == 200:
    data = response.json()
    if data['status'] == '000':
        for report in data['list']:
            if '사업보고서' in report['report_nm']:  # 사업보고서 필터링
                print(f"📌 사업보고서 발견: {report['report_nm']}, 접수일: {report['rcept_dt']}, 문서번호: {report['rcept_no']}")
                # 위에서 출력된 문서번호를 사용하여 다운로드
    else:
        print(f"❌ 공시 목록 조회 실패: {data['message']}")
else:
    print(f"❌ API 요청 실패: {response.status_code}")


📌 사업보고서 발견: [기재정정]사업보고서 (2022.12), 접수일: 20230711, 문서번호: 20230711000155
📌 사업보고서 발견: 사업보고서 (2022.12), 접수일: 20230320, 문서번호: 20230320000880


In [25]:
# 정확한 문서번호 사용
DOC_NO = '20230330001115호'

# DART API로 XML 다운로드
DOC_URL = f'https://opendart.fss.or.kr/api/document.xml?crtfc_key={API_KEY}&rcept_no={DOC_NO}'

# XML 파일 다운로드
response = requests.get(DOC_URL)

if response.status_code == 200:
    with open('business_report.xml', 'wb') as file:
        file.write(response.content)
    print('✅ 사업보고서 XML 다운로드 완료: business_report.xml')
else:
    print(f'❌ XML 다운로드 실패: {response.status_code}')


✅ 사업보고서 XML 다운로드 완료: business_report.xml


In [29]:
import requests

# DART API 인증키
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'

# 문서번호 (위에서 얻은 문서번호 사용)
DOC_NO = '20230711000155'

# DART API로 XML 다운로드
DOC_URL = f'https://opendart.fss.or.kr/api/document.xml?crtfc_key={API_KEY}&rcept_no={DOC_NO}'

# XML 파일 다운로드
response = requests.get(DOC_URL)

if response.status_code == 200:
    with open('business_report.xml', 'wb') as file:
        file.write(response.content)
    print('✅ 사업보고서 XML 다운로드 완료: business_report.xml')
else:
    print(f'❌ XML 다운로드 실패: {response.status_code}')


✅ 사업보고서 XML 다운로드 완료: business_report.xml


In [30]:
!pip install pandas lxml

In [34]:
import pandas as pd
from lxml import etree

# XBRL 파일 경로
xbrl_file = '/content/entity01158632_2023-12-31.xbrl'

# XBRL 파일 파싱
tree = etree.parse(xbrl_file)
root = tree.getroot()

# 네임스페이스 처리 (XBRL에서 네임스페이스가 중요한 경우가 많음)
namespaces = {
    'xbrl': 'http://www.xbrl.org/2003/instance',
    'link': 'http://www.xbrl.org/2003/linkbase',
}

# XBRL에서 특정 태그 찾기 (예: 재무제표 정보)
# 예시로, 'xbrl:context'와 'xbrl:unit'을 찾아보겠습니다.
contexts = root.xpath('//xbrl:context', namespaces=namespaces)
units = root.xpath('//xbrl:unit', namespaces=namespaces)

# 'xbrl:context'의 ID와 'xbrl:unit'의 ID를 추출하여 데이터프레임 만들기
data = []

for context in contexts:
    context_id = context.get('id')
    for unit in units:
        unit_id = unit.get('id')
        data.append({
            'context_id': context_id,
            'unit_id': unit_id,
        })

# pandas 데이터프레임으로 변환
df = pd.DataFrame(data)

# 데이터프레임 출력
print(df.head())


                                          context_id unit_id
0  CFY2023dFY_dart-gcd_PeriodAxis_dart-gcd_Period...  KRWEPS
1  CFY2023dFY_dart-gcd_PeriodAxis_dart-gcd_Period...     KRW
2  CFY2023dFY_dart-gcd_PeriodAxis_dart-gcd_Period...    PURE
3  CFY2023dFY_ifrs-full_ConsolidatedAndSeparateFi...  KRWEPS
4  CFY2023dFY_ifrs-full_ConsolidatedAndSeparateFi...     KRW


In [45]:
import requests

# DART API 인증키
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'

# 기업 고유번호 (corp_code) 입력
CORP_CODE = '01359295'  # 예시: '00126380'

# 공시 조회 시작과 끝 날짜 설정 (2024년 전체)
START_DATE = '20240101'
END_DATE = '20241231'

# DART API URL (기업 코드 및 공시 목록 조회)
URL = f'https://opendart.fss.or.kr/api/list.json?crtfc_key={API_KEY}&corp_code={CORP_CODE}&bgn_de={START_DATE}&end_de={END_DATE}&page_no=1&page_count=10'

# API 요청
response = requests.get(URL)

# 응답 처리
if response.status_code == 200:
    data = response.json()
    if data['status'] == '000':
        # 감사보고서만 필터링
        audit_report_found = False

        for report in data['list']:
            report_name = report['report_nm']
            # "감사보고서"가 포함된 문서 찾기
            if '감사보고서' in report_name:
                print(f"📌 감사보고서 발견: {report_name}, 접수일: {report['rcept_dt']}, 문서번호: {report['rcept_no']}")
                # 문서번호를 사용하여 다운로드
                doc_no = report['rcept_no']
                DOC_URL = f'https://opendart.fss.or.kr/api/document.xml?crtfc_key={API_KEY}&rcept_no={doc_no}'
                doc_response = requests.get(DOC_URL)
                if doc_response.status_code == 200:
                    with open(f'audit_report_{doc_no}.xml', 'wb') as file:
                        file.write(doc_response.content)
                    print(f'✅ 감사보고서 XML 다운로드 완료: audit_report_{doc_no}.xml')
                else:
                    print(f'❌ 감사보고서 XML 다운로드 실패: {doc_response.status_code}')
                audit_report_found = True

        if not audit_report_found:
            print("❌ 2024년에 제출된 감사보고서가 없습니다.")
    else:
        print(f"❌ 공시 목록 조회 실패: {data['message']}")
else:
    print(f"❌ API 요청 실패: {response.status_code}")


📌 감사보고서 발견: 연결감사보고서 (2023.12), 접수일: 20240412, 문서번호: 20240412001153
✅ 감사보고서 XML 다운로드 완료: audit_report_20240412001153.xml
📌 감사보고서 발견: 감사보고서 (2023.12), 접수일: 20240412, 문서번호: 20240412001149
✅ 감사보고서 XML 다운로드 완료: audit_report_20240412001149.xml


In [46]:
import requests

# DART API 인증키
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'

# 문서번호 (이미 확인된 문서 번호 사용)
DOC_NO = '20240412001149'  # 예시 문서 번호

# 문서 다운로드 URL
DOC_URL = f'https://opendart.fss.or.kr/api/document.xml?crtfc_key={API_KEY}&rcept_no={DOC_NO}'

# XML 파일 다운로드 요청
response = requests.get(DOC_URL)

# 응답 상태 코드 및 내용 확인
if response.status_code == 200:
    print(f"✅ 요청 성공! 응답 상태 코드: {response.status_code}")
    # 응답 본문 확인 (첫 500자 출력)
    print("응답 내용 (최초 500자):", response.text[:500])

    if response.text.strip() == "":
        print("❌ 응답 내용이 비어있습니다. 문서 번호를 다시 확인해보세요.")
    else:
        # 응답 내용이 있다면 파일로 저장
        with open(f'audit_report_{DOC_NO}.xml', 'wb') as file:
            file.write(response.content)
        print(f'✅ XML 파일 다운로드 완료: audit_report_{DOC_NO}.xml')
else:
    print(f"❌ 요청 실패: 상태 코드 {response.status_code}, 응답 내용: {response.text}")


✅ 요청 성공! 응답 상태 코드: 200
응답 내용 (최초 500자): PK   �?��               20240412001149_00760.xml�ksW�(��#��KĄ�����G�	lp7w0w�?9:f����3='��ޙO'
�0m7�B�ԙ���������A�����~_7�ɧ|��g0����v���hAl?g?=w��
✅ XML 파일 다운로드 완료: audit_report_20240412001149.xml


In [47]:
import zipfile
import os

# 다운로드한 파일 경로
zip_file_path = 'audit_report_20240412001149.xml'
extracted_folder = 'extracted_report/'

# 압축 파일을 풀 폴더 경로 설정
os.makedirs(extracted_folder, exist_ok=True)

# ZIP 파일 압축 해제
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)

print(f"✅ 압축 해제 완료. 파일은 {extracted_folder} 폴더에 저장되었습니다.")



✅ 압축 해제 완료. 파일은 extracted_report/ 폴더에 저장되었습니다.


In [52]:
import xml.etree.ElementTree as ET
import pandas as pd

# XML 파일 경로
xml_file_path = '/content/extracted_report/20240412001149_00760.xml'  # XML 파일 경로를 지정

# XML 파일 파싱
tree = ET.parse(xml_file_path)
root = tree.getroot()

# 재무제표 추출 함수
def extract_financial_statements(root):
    financial_data = {
        '항목': [],
        '금액': []
    }

    # 모든 <TR> 태그를 순회하면서 재무 항목을 추출
    for tr in root.iter('TR'):
        td_list = list(tr.iter('TD'))

        # <TD> 태그가 3개 이상인 경우에만 처리
        if len(td_list) >= 3:
            항목 = td_list[0].text.strip() if td_list[0].text else None  # 첫 번째 <TD> 태그에서 항목
            금액 = td_list[2].text.strip() if td_list[2].text else None  # 세 번째 <TD> 태그에서 금액

            # 항목과 금액이 모두 존재하는 경우에만 추가
            if 항목 and 금액:
                financial_data['항목'].append(항목)
                financial_data['금액'].append(금액)

    return financial_data

# 재무제표 정보 추출
financial_statements = extract_financial_statements(root)

# pandas DataFrame으로 변환
df = pd.DataFrame(financial_statements)

# DataFrame 출력
print(df)
df.to_csv('test.csv',encoding='utf-8-sig')

               항목                            금액
0        본점 소재지 :  서울특별시 구로구 디지털로 243 지하이시티 19층
1        현금및현금성자산                   476,743,848
2       매출채권및기타채권                 3,446,764,277
3          기타유동자산                   174,876,130
4         당기법인세자산                    15,219,710
..            ...                           ...
610             일                             명
611    재고자산실사(입회)                             일
612    금융자산실사(입회)                             일
613          외부조회                        채권채무조회
614  지배기구와의커뮤니케이션                             회

[615 rows x 2 columns]


In [53]:
def extract_financial_statements(root):
    financial_data = {}

    # 예시: 'totalRevenue'와 'netIncome' 태그를 찾는 방식
    for item in root.iter('XBRL'):  # XBRL 태그 예시
        for child in item:
            tag = child.tag
            text = child.text

            # 예시: 실제 태그 이름을 기반으로 재무정보를 추출
            if tag == 'totalRevenue':  # 매출액 태그 예시
                financial_data['매출액'] = text
            elif tag == 'netIncome':  # 순이익 태그 예시
                financial_data['순이익'] = text

    return financial_data


In [55]:
df.to_csv('ztest.csv', encoding='utf-8-sig')

In [56]:
import requests
import zipfile
import os
import xml.etree.ElementTree as ET
import pandas as pd

# DART API 인증키
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'

# 문서번호 (이미 확인된 문서 번호 사용)
DOC_NO = '20240412001149'  # 예시 문서 번호

# 문서 다운로드 URL
DOC_URL = f'https://opendart.fss.or.kr/api/document.xml?crtfc_key={API_KEY}&rcept_no={DOC_NO}'

# XML 파일 다운로드 요청
response = requests.get(DOC_URL)

# 응답 상태 코드 및 내용 확인
if response.status_code == 200:
    print(f"✅ 요청 성공! 응답 상태 코드: {response.status_code}")

    # 응답 본문 확인 (첫 500자 출력)
    if response.text.strip() == "":
        print("❌ 응답 내용이 비어있습니다. 문서 번호를 다시 확인해보세요.")
    else:
        # 응답 내용이 있다면 파일로 저장
        zip_file_path = f'audit_report_{DOC_NO}.xml'
        with open(zip_file_path, 'wb') as file:
            file.write(response.content)
        print(f'✅ XML 파일 다운로드 완료: {zip_file_path}')

        # 파일이 ZIP 파일인지 확인하고 압축 해제
        extracted_folder = 'extracted_report/'
        os.makedirs(extracted_folder, exist_ok=True)

        # 파일이 ZIP 파일인지 체크
        if zip_file_path.endswith('.zip'):
            try:
                with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                    zip_ref.extractall(extracted_folder)
                print(f"✅ 압축 해제 완료. 파일은 {extracted_folder} 폴더에 저장되었습니다.")
                # 압축 해제된 파일 경로
                xml_file_path = os.path.join(extracted_folder, zip_ref.namelist()[0])
            except zipfile.BadZipFile:
                print("❌ ZIP 파일이 아닙니다.")
                xml_file_path = zip_file_path
        else:
            xml_file_path = zip_file_path

        # XML 파일 파싱
        tree = ET.parse(xml_file_path)
        root = tree.getroot()

        # 재무제표 추출 함수
        def extract_financial_statements(root):
            financial_data = {
                '항목': [],
                '금액': []
            }

            # 모든 <TR> 태그를 순회하면서 재무 항목을 추출
            for tr in root.iter('TR'):
                td_list = list(tr.iter('TD'))

                # <TD> 태그가 3개 이상인 경우에만 처리
                if len(td_list) >= 3:
                    항목 = td_list[0].text.strip() if td_list[0].text else None  # 첫 번째 <TD> 태그에서 항목
                    금액 = td_list[2].text.strip() if td_list[2].text else None  # 세 번째 <TD> 태그에서 금액

                    # 항목과 금액이 모두 존재하는 경우에만 추가
                    if 항목 and 금액:
                        financial_data['항목'].append(항목)
                        financial_data['금액'].append(금액)

            return financial_data

        # 재무제표 정보 추출
        financial_statements = extract_financial_statements(root)

        # pandas DataFrame으로 변환
        df = pd.DataFrame(financial_statements)

        # DataFrame 출력
        print(df)

        # CSV 파일로 저장
        df.to_csv(f'test_{DOC_NO}.csv', encoding='utf-8-sig')
        print(f'✅ CSV 파일로 저장 완료: test_{DOC_NO}.csv')
else:
    print(f"❌ 요청 실패: 상태 코드 {response.status_code}, 응답 내용: {response.text}")


✅ 요청 성공! 응답 상태 코드: 200
✅ XML 파일 다운로드 완료: audit_report_20240412001149.xml


ParseError: not well-formed (invalid token): line 1, column 2 (<string>)

In [58]:
import zipfile
import os
import xml.etree.ElementTree as ET

# ZIP 파일을 추출하고 XML 파일을 확인
def extract_and_parse_zip(zip_file_path, extracted_folder):
    # ZIP 파일이 있다면 압축을 해제합니다.
    os.makedirs(extracted_folder, exist_ok=True)

    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_folder)

    # 압축을 해제한 파일을 찾습니다.
    extracted_files = os.listdir(extracted_folder)
    print(f"✅ 압축 해제된 파일 목록: {extracted_files}")

    # 추출된 첫 번째 XML 파일을 파싱합니다.
    for file in extracted_files:
        if file.endswith('.xml'):  # XML 파일인지 확인
            xml_file_path = os.path.join(extracted_folder, file)
            print(f"✅ 파싱할 XML 파일: {xml_file_path}")
            return xml_file_path
    return None  # XML 파일이 없으면 None 반환

# XML 파일 파싱
def parse_xml(xml_file_path):
    try:
        tree = ET.parse(xml_file_path)
        root = tree.getroot()
        return root
    except ET.ParseError as e:
        print(f"❌ XML 파싱 오류: {e}")
        return None

# XML 파일 경로
xml_file_path = 'audit_report_20240412001149.xml'  # 다운로드한 파일 경로
extracted_folder = 'extracted_report/'

# 압축을 풀고 XML 파일 파싱
xml_file_path = extract_and_parse_zip(xml_file_path, extracted_folder)

if xml_file_path:
    # XML 파일 파싱
    root = parse_xml(xml_file_path)

    if root:
        print(f"✅ XML 파싱 성공, 루트 요소: {root.tag}")
else:
    print("❌ XML 파일을 찾을 수 없습니다.")


✅ 압축 해제된 파일 목록: ['.ipynb_checkpoints', '20240412001149_00760.xml', '20240401000134_00760.xml']
✅ 파싱할 XML 파일: extracted_report/20240412001149_00760.xml
✅ XML 파싱 성공, 루트 요소: DOCUMENT


In [1]:
import requests
import zipfile
import os
import xml.etree.ElementTree as ET
import pandas as pd

# ✅ DART API 인증키
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'  # 실제 키로 변경 필요
CORP_CODE = '01359295'  # 기업 코드 (예제: 삼성전자)
START_DATE = '20240101'
END_DATE = '20241231'

# ✅ 1. DART API에서 감사보고서 문서번호 가져오기
URL = f'https://opendart.fss.or.kr/api/list.json?crtfc_key={API_KEY}&corp_code={CORP_CODE}&bgn_de={START_DATE}&end_de={END_DATE}&page_no=1&page_count=10'
response = requests.get(URL)

if response.status_code == 200:
    data = response.json()
    if data['status'] == '000':
        for report in data['list']:
            if '감사보고서' in report['report_nm']:
                DOC_NO = report['rcept_no']
                print(f"📌 감사보고서 발견: {report['report_nm']}, 접수일: {report['rcept_dt']}, 문서번호: {DOC_NO}")
                break
        else:
            print("❌ 2024년에 제출된 감사보고서가 없습니다.")
            exit()
    else:
        print(f"❌ 공시 목록 조회 실패: {data['message']}")
        exit()
else:
    print(f"❌ API 요청 실패: {response.status_code}")
    exit()

# ✅ 2. 문서 다운로드 및 저장
DOC_URL = f'https://opendart.fss.or.kr/api/document.xml?crtfc_key={API_KEY}&rcept_no={DOC_NO}'
response = requests.get(DOC_URL)

if response.status_code == 200 and response.content.strip():
    zip_file_path = f'audit_report_{DOC_NO}.zip'
    with open(zip_file_path, 'wb') as file:
        file.write(response.content)
    print(f"✅ XML ZIP 다운로드 완료: {zip_file_path}")
else:
    print("❌ XML 다운로드 실패 또는 파일이 비어 있습니다.")
    exit()

# ✅ 3. ZIP 파일 압축 해제
extracted_folder = 'extracted_report/'
os.makedirs(extracted_folder, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)

# ✅ 4. 압축 해제된 XML 파일 확인
extracted_files = os.listdir(extracted_folder)
xml_file_path = None
for file in extracted_files:
    if file.endswith('.xml'):
        xml_file_path = os.path.join(extracted_folder, file)
        print(f"✅ 추출된 XML 파일: {xml_file_path}")
        break

if not xml_file_path:
    print("❌ XML 파일을 찾을 수 없습니다.")
    exit()

# ✅ 5. XML 파일 파싱하여 재무제표 추출
def extract_financial_statements(xml_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    financial_data = {'항목': [], '금액': []}

    # <TR> 태그를 순회하며 데이터 추출
    for tr in root.iter('TR'):
        td_list = list(tr.iter('TD'))

        if len(td_list) >= 3:  # 3개 이상의 <TD> 태그가 있는 경우
            항목 = td_list[0].text.strip() if td_list[0].text else None
            금액 = td_list[2].text.strip() if td_list[2].text else None

            if 항목 and 금액:
                financial_data['항목'].append(항목)
                financial_data['금액'].append(금액)

    return financial_data

# ✅ 6. 재무제표 DataFrame 생성
financial_statements = extract_financial_statements(xml_file_path)
df = pd.DataFrame(financial_statements)

if not df.empty:
    print("✅ 추출된 재무제표:")
    print(df.head())

    # ✅ 7. Excel로 저장
    output_excel = f'financial_statements_{DOC_NO}.xlsx'
df.to_excel(output_excel, index=False)  # encoding 제거
print(f"✅ 재무제표 엑셀 저장 완료: {output_excel}")

📌 감사보고서 발견: 연결감사보고서 (2023.12), 접수일: 20240412, 문서번호: 20240412001153
✅ XML ZIP 다운로드 완료: audit_report_20240412001153.zip
✅ 추출된 XML 파일: extracted_report/20240412001153_00761.xml
✅ 추출된 재무제표:
          항목                            금액
0   본점 소재지 :  서울특별시 구로구 디지털로 243 지하이시티 19층
1   현금및현금성자산                 1,200,161,722
2  매출채권및기타채권                 2,207,037,860
3     기타유동자산                   174,876,130
4    당기법인세자산                    15,219,710
✅ 재무제표 엑셀 저장 완료: financial_statements_20240412001153.xlsx


In [17]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import os
import zipfile

# ✅ 1. DART API 설정
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'  # 실제 키로 변경 필요
CORP_CODE = '01359295'  # 기업 코드 (예제: 삼성전자)
START_DATE = '20240101'
END_DATE = '20241231'


# ✅ 2. 공시 목록 조회 (감사보고서 필터링)
URL = f'https://opendart.fss.or.kr/api/list.json?crtfc_key={API_KEY}&corp_code={CORP_CODE}&bgn_de={START_DATE}&end_de={END_DATE}&page_no=1&page_count=20'
response = requests.get(URL)
data = response.json()

if data['status'] == '000':
    doc_list = data.get('list', [])

    # 🔍 "연결감사보고서"가 아닌 "감사보고서"만 찾기
    audit_doc = next((doc for doc in doc_list if '감사보고서' in doc['report_nm'] and '연결' not in doc['report_nm']), None)

    if audit_doc:
        DOC_NO = audit_doc['rcept_no']
        print(f"📌 감사보고서 발견: {audit_doc['report_nm']}, 접수일: {audit_doc['rcept_dt']}, 문서번호: {DOC_NO}")

        # ✅ 3. 문서 다운로드
        DOC_URL = f'https://opendart.fss.or.kr/api/document.xml?crtfc_key={API_KEY}&rcept_no={DOC_NO}'
        doc_response = requests.get(DOC_URL)

        if doc_response.status_code == 200:
            zip_file_path = f'audit_report_{DOC_NO}.zip'
            with open(zip_file_path, 'wb') as file:
                file.write(doc_response.content)
            print(f'✅ XML 압축 파일 다운로드 완료: {zip_file_path}')

            # ✅ 4. ZIP 파일 해제
            extracted_folder = f'extracted_report_{DOC_NO}/'
            os.makedirs(extracted_folder, exist_ok=True)

            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(extracted_folder)
            print(f"✅ 압축 해제 완료: {extracted_folder}")

            # ✅ 5. XML 파싱 및 재무제표 추출
            xml_files = [f for f in os.listdir(extracted_folder) if f.endswith('.xml')]
            if xml_files:
                xml_file_path = os.path.join(extracted_folder, xml_files[0])

                tree = ET.parse(xml_file_path)
                root = tree.getroot()

                # 재무제표 추출 함수
                def extract_financial_statements(root):
                    financial_data = {'항목': [], '금액': []}
                    for tr in root.iter('TR'):
                        td_list = list(tr.iter('TD'))
                        if len(td_list) >= 3:
                            항목 = td_list[0].text.strip() if td_list[0].text else None
                            금액 = td_list[2].text.strip() if td_list[2].text else None
                            if 항목 and 금액:
                                financial_data['항목'].append(항목)
                                financial_data['금액'].append(금액)
                    return financial_data

                financial_statements = extract_financial_statements(root)
                df = pd.DataFrame(financial_statements)

                # ✅ 6. 엑셀 저장
                output_excel = f'financial_statements_{DOC_NO}.xlsx'
                df.to_excel(output_excel, index=False)
                print(f"✅ 재무제표 엑셀 저장 완료: {output_excel}")
            else:
                print("❌ XML 파일이 없습니다.")
        else:
            print(f'❌ XML 다운로드 실패: {doc_response.status_code}')
    else:
        print("❌ 2024년에 제출된 개별 감사보고서가 없습니다.")
else:
    print(f"❌ 공시 목록 조회 실패: {data['message']}")


📌 감사보고서 발견: 감사보고서 (2023.12), 접수일: 20240412, 문서번호: 20240412001149
✅ XML 압축 파일 다운로드 완료: audit_report_20240412001149.zip
✅ 압축 해제 완료: extracted_report_20240412001149/
✅ 재무제표 엑셀 저장 완료: financial_statements_20240412001149.xlsx


In [24]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import os
import zipfile

# ✅ 1. DART API 설정
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'  # 실제 키로 변경 필요
CORP_CODE = '01359295'  # 기업 코드 (예제: 삼성전자)
START_DATE = '20240101'
END_DATE = '20241231'

# ✅ 2. 공시 목록 조회 (감사보고서 필터링)
URL = f'https://opendart.fss.or.kr/api/list.json?crtfc_key={API_KEY}&corp_code={CORP_CODE}&bgn_de={START_DATE}&end_de={END_DATE}&page_no=1&page_count=20'
response = requests.get(URL)
data = response.json()

if data['status'] == '000':
    doc_list = data.get('list', [])

    # 🔍 "연결감사보고서"가 아닌 "감사보고서"만 찾기
    audit_doc = next((doc for doc in doc_list if '감사보고서' in doc['report_nm'] and '연결' not in doc['report_nm']), None)

    if audit_doc:
        DOC_NO = audit_doc['rcept_no']
        print(f"📌 감사보고서 발견: {audit_doc['report_nm']}, 접수일: {audit_doc['rcept_dt']}, 문서번호: {DOC_NO}")

        # ✅ 3. 문서 다운로드
        DOC_URL = f'https://opendart.fss.or.kr/api/document.xml?crtfc_key={API_KEY}&rcept_no={DOC_NO}'
        doc_response = requests.get(DOC_URL)

        if doc_response.status_code == 200:
            zip_file_path = f'audit_report_{DOC_NO}.zip'
            with open(zip_file_path, 'wb') as file:
                file.write(doc_response.content)
            print(f'✅ XML 압축 파일 다운로드 완료: {zip_file_path}')

            # ✅ 4. ZIP 파일 해제
            extracted_folder = f'extracted_report_{DOC_NO}/'
            os.makedirs(extracted_folder, exist_ok=True)

            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(extracted_folder)
            print(f"✅ 압축 해제 완료: {extracted_folder}")

            # ✅ 5. XML 파싱 및 재무제표 추출
            xml_files = [f for f in os.listdir(extracted_folder) if f.endswith('.xml')]
            if xml_files:
                xml_file_path = os.path.join(extracted_folder, xml_files[0])

                tree = ET.parse(xml_file_path)
                root = tree.getroot()

                # 재무제표 추출 함수 (주석 항목 제외, 당기와 전기 구분)
                def extract_financial_statements(root):
                    financial_data = {'항목': [], '당기 금액': [], '전기 금액': []}
                    seen_items = set()  # 중복을 확인할 집합 (set)

                    for tr in root.iter('TR'):
                        td_list = list(tr.iter('TD'))
                        if len(td_list) >= 3:
                            항목 = td_list[0].text.strip() if td_list[0].text else None
                            당기금액 = td_list[2].text.strip() if td_list[2].text else None

                            # 전기금액이 있을 경우 7번째 열을 확인
                            if len(td_list) > 6:
                                전기금액 = td_list[6].text.strip() if td_list[6].text else None
                            else:
                                전기금액 = None

                            if 항목 and (당기금액 or 전기금액) and '주석' not in 항목 and 항목 != '':
                                # 항목이 중복되지 않은 경우에만 추가
                                if 항목 not in seen_items:
                                    seen_items.add(항목)
                                    financial_data['항목'].append(항목)
                                    financial_data['당기 금액'].append(당기금액 if 당기금액 else '없음')
                                    financial_data['전기 금액'].append(전기금액 if 전기금액 else '없음')

                    return financial_data

                financial_statements = extract_financial_statements(root)
                df = pd.DataFrame(financial_statements)

                # ✅ 6. 엑셀 저장
                output_excel = f'financial_statements_{DOC_NO}.xlsx'
                df.to_excel(output_excel, index=False)
                print(f"✅ 재무제표 엑셀 저장 완료: {output_excel}")
            else:
                print("❌ XML 파일이 없습니다.")
        else:
            print(f'❌ XML 다운로드 실패: {doc_response.status_code}')
    else:
        print("❌ 2024년에 제출된 개별 감사보고서가 없습니다.")
else:
    print(f"❌ 공시 목록 조회 실패: {data['message']}")


📌 감사보고서 발견: 감사보고서 (2023.12), 접수일: 20240412, 문서번호: 20240412001149
✅ XML 압축 파일 다운로드 완료: audit_report_20240412001149.zip
✅ 압축 해제 완료: extracted_report_20240412001149/
✅ 재무제표 엑셀 저장 완료: financial_statements_20240412001149.xlsx


In [26]:
import requests
import xml.etree.ElementTree as ET
import os
import zipfile

# ✅ 1. DART API 설정
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'  # 실제 키로 변경 필요
CORP_CODE = '01359295'  # 기업 코드 (예제: 삼성전자)
START_DATE = '20240101'
END_DATE = '20241231'

# ✅ 2. 공시 목록 조회 (감사보고서 필터링)
URL = f'https://opendart.fss.or.kr/api/list.json?crtfc_key={API_KEY}&corp_code={CORP_CODE}&bgn_de={START_DATE}&end_de={END_DATE}&page_no=1&page_count=20'
response = requests.get(URL)
data = response.json()

if data['status'] == '000':
    doc_list = data.get('list', [])

    # 🔍 "연결감사보고서"가 아닌 "감사보고서"만 찾기
    audit_doc = next((doc for doc in doc_list if '감사보고서' in doc['report_nm'] and '연결' not in doc['report_nm']), None)

    if audit_doc:
        DOC_NO = audit_doc['rcept_no']
        print(f"📌 감사보고서 발견: {audit_doc['report_nm']}, 접수일: {audit_doc['rcept_dt']}, 문서번호: {DOC_NO}")

        # ✅ 3. 문서 다운로드
        DOC_URL = f'https://opendart.fss.or.kr/api/document.xml?crtfc_key={API_KEY}&rcept_no={DOC_NO}'
        doc_response = requests.get(DOC_URL)

        if doc_response.status_code == 200:
            zip_file_path = f'audit_report_{DOC_NO}.zip'
            with open(zip_file_path, 'wb') as file:
                file.write(doc_response.content)
            print(f'✅ XML 압축 파일 다운로드 완료: {zip_file_path}')

            # ✅ 4. ZIP 파일 해제
            extracted_folder = f'extracted_report_{DOC_NO}/'
            os.makedirs(extracted_folder, exist_ok=True)

            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(extracted_folder)
            print(f"✅ 압축 해제 완료: {extracted_folder}")

            # ✅ 5. XML 파싱 및 재무제표 추출
            xml_files = [f for f in os.listdir(extracted_folder) if f.endswith('.xml')]
            if xml_files:
                xml_file_path = os.path.join(extracted_folder, xml_files[0])

                tree = ET.parse(xml_file_path)
                root = tree.getroot()

                # 재무제표 추출 함수 (트리 형태로 추출)
                def extract_financial_tree(root):
                    financial_tree = {}
                    current_parent = None  # 현재 부모 항목을 추적
                    for tr in root.iter('TR'):
                        td_list = list(tr.iter('TD'))
                        if len(td_list) >= 4:
                            항목 = td_list[0].text.strip() if td_list[0].text else None
                            금액 = td_list[3].text.strip() if td_list[3].text else None  # 당기 금액

                            if 항목 and 금액 and '주석' not in 항목:
                                if "자산" in 항목:  # 자산 항목이 나왔을 때
                                    current_parent = 항목
                                    if current_parent not in financial_tree:
                                        financial_tree[current_parent] = {}

                                elif "유동자산" in 항목 and current_parent == "자산":  # 유동자산 항목
                                    current_parent = 항목
                                    financial_tree["자산"][current_parent] = {}

                                elif "현금및현금성자산" in 항목 and current_parent == "유동자산":  # 현금및현금성자산 항목
                                    financial_tree["자산"]["유동자산"][항목] = 금액

                    return financial_tree

                financial_tree = extract_financial_tree(root)
                print("✅ 추출된 재무제표 트리:")
                print(financial_tree)

            else:
                print("❌ XML 파일이 없습니다.")
        else:
            print(f'❌ XML 다운로드 실패: {doc_response.status_code}')
    else:
        print("❌ 2024년에 제출된 개별 감사보고서가 없습니다.")
else:
    print(f"❌ 공시 목록 조회 실패: {data['message']}")


📌 감사보고서 발견: 감사보고서 (2023.12), 접수일: 20240412, 문서번호: 20240412001149
✅ XML 압축 파일 다운로드 완료: audit_report_20240412001149.zip
✅ 압축 해제 완료: extracted_report_20240412001149/
✅ 추출된 재무제표 트리:
{'I. 유동자산': {}, 'II. 비유동자산': {}, '자산총계': {}, '무형자산상각비': {}, '재고자산평가손실': {}, '재고자산감모손실': {}, '유형자산처분이익': {}, '당기손익-공정가치측정금융자산처분이익': {}, '재고자산의 감소(증가)': {}, '당기손익공정가치측정금융자산의 처분': {}, '당기손익-공정가치측정금융자산의 취득': {}, '건설중인자산의 취득': {}, '현금및현금성자산': {}, '차감: 현금및현금성자산': {}, '공정가치측정금융자산': {}, '파생상품자산': {}, '상각후원가측정금융자산': {}, '기타금융자산': {}, '기타비유동금융자산': {}, '당기손익-공정가치측정금융자산': {}, '건설중인자산': {}, '무형자산감가상각비': {}, '리스자산': {}, '이연법인세자산(부채) 합계': {}, '이연법인세자산 미인식액': {}, '12개월 이내에 회수될 이연법인세자산': {}, '12개월 이후에 회수될 이연법인세자산': {}, '이연법인세자산(부채) 순액': {}, '이연법인세자산(부채) 인식액': {}, '유무형자산(*2)': {}, '파생상품자산(*7)': {}}


In [29]:
import requests
import xml.etree.ElementTree as ET
import os
import zipfile
import pandas as pd

# ✅ 1. DART API 설정
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'  # 실제 키로 변경 필요
CORP_CODE = '01359295'  # 기업 코드 (예제: 삼성전자)
START_DATE = '20240101'
END_DATE = '20241231'

# ✅ 2. 공시 목록 조회 (감사보고서 필터링)
URL = f'https://opendart.fss.or.kr/api/list.json?crtfc_key={API_KEY}&corp_code={CORP_CODE}&bgn_de={START_DATE}&end_de={END_DATE}&page_no=1&page_count=20'
response = requests.get(URL)
data = response.json()

if data['status'] == '000':
    doc_list = data.get('list', [])

    # 🔍 "연결감사보고서"가 아닌 "감사보고서"만 찾기
    audit_doc = next((doc for doc in doc_list if '감사보고서' in doc['report_nm'] and '연결' not in doc['report_nm']), None)

    if audit_doc:
        DOC_NO = audit_doc['rcept_no']
        print(f"📌 감사보고서 발견: {audit_doc['report_nm']}, 접수일: {audit_doc['rcept_dt']}, 문서번호: {DOC_NO}")

        # ✅ 3. 문서 다운로드
        DOC_URL = f'https://opendart.fss.or.kr/api/document.xml?crtfc_key={API_KEY}&rcept_no={DOC_NO}'
        doc_response = requests.get(DOC_URL)

        if doc_response.status_code == 200:
            zip_file_path = f'audit_report_{DOC_NO}.zip'
            with open(zip_file_path, 'wb') as file:
                file.write(doc_response.content)
            print(f'✅ XML 압축 파일 다운로드 완료: {zip_file_path}')

            # ✅ 4. ZIP 파일 해제
            extracted_folder = f'extracted_report_{DOC_NO}/'
            os.makedirs(extracted_folder, exist_ok=True)

            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(extracted_folder)
            print(f"✅ 압축 해제 완료: {extracted_folder}")

            # ✅ 5. XML 파싱 및 재무제표 추출
            xml_files = [f for f in os.listdir(extracted_folder) if f.endswith('.xml')]
            if xml_files:
                xml_file_path = os.path.join(extracted_folder, xml_files[0])

                tree = ET.parse(xml_file_path)
                root = tree.getroot()

                # 재무제표 트리 구조 추출 함수 (당기, 전기 포함)
                def extract_financial_tree(root):
                    financial_tree = []
                    current_parent = None  # 현재 부모 항목을 추적
                    for tr in root.iter('TR'):
                        td_list = list(tr.iter('TD'))
                        if len(td_list) >= 8:  # 8개 열이 존재해야 제대로 된 정보가 있음
                            항목 = td_list[0].text.strip() if td_list[0].text else None
                            당기금액 = td_list[3].text.strip() if td_list[3].text else None  # 당기 금액
                            전기금액 = td_list[6].text.strip() if len(td_list) > 6 and td_list[6].text else None  # 전기 금액

                            if 항목 and (당기금액 or 전기금액) and '주석' not in 항목:
                                # 자산 항목을 추출하고 트리 구조에 추가
                                if "자산" in 항목:
                                    current_parent = 항목
                                    financial_tree.append([current_parent, 항목, 당기금액, 전기금액])

                                # 유동자산 항목을 추출하고 자산 하위에 추가
                                elif "유동자산" in 항목 and current_parent == "자산":
                                    current_parent = 항목
                                    financial_tree.append([current_parent, 항목, 당기금액, 전기금액])

                                # 현금및현금성자산 항목을 추출하고 유동자산 하위에 추가
                                elif "현금및현금성자산" in 항목 and current_parent == "유동자산":
                                    financial_tree.append([current_parent, 항목, 당기금액, 전기금액])

                    return financial_tree

                # 재무제표 트리 추출
                financial_tree = extract_financial_tree(root)

                # ✅ 6. 데이터프레임으로 변환
                df = pd.DataFrame(financial_tree, columns=['상위 항목', '하위 항목', '당기 금액', '전기 금액'])

                # ✅ 7. 엑셀 저장
                output_excel = f'financial_statements_{DOC_NO}.xlsx'
                df.to_excel(output_excel, index=False)
                print(f"✅ 재무제표 엑셀 저장 완료: {output_excel}")

            else:
                print("❌ XML 파일이 없습니다.")
        else:
            print(f'❌ XML 다운로드 실패: {doc_response.status_code}')
    else:
        print("❌ 2024년에 제출된 개별 감사보고서가 없습니다.")
else:
    print(f"❌ 공시 목록 조회 실패: {data['message']}")


📌 감사보고서 발견: 감사보고서 (2023.12), 접수일: 20240412, 문서번호: 20240412001149
✅ XML 압축 파일 다운로드 완료: audit_report_20240412001149.zip
✅ 압축 해제 완료: extracted_report_20240412001149/
✅ 재무제표 엑셀 저장 완료: financial_statements_20240412001149.xlsx


In [33]:
import requests
import xml.etree.ElementTree as ET
import os
import zipfile
import pandas as pd

# ✅ 1. DART API 설정
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'  # 실제 키로 변경 필요
CORP_CODE = '01359295'  # 기업 코드 (예: 삼성전자)
START_DATE = '20240101'
END_DATE = '20241231'

# ✅ 2. 공시 목록 조회 (감사보고서 필터링)
URL = f'https://opendart.fss.or.kr/api/list.json?crtfc_key={API_KEY}&corp_code={CORP_CODE}&bgn_de={START_DATE}&end_de={END_DATE}&page_no=1&page_count=20'
response = requests.get(URL)
data = response.json()

if data['status'] == '000':
    doc_list = data.get('list', [])

    # 🔍 "연결감사보고서"가 아닌 "감사보고서"만 찾기
    audit_doc = next((doc for doc in doc_list if '감사보고서' in doc['report_nm'] and '연결' not in doc['report_nm']), None)

    if audit_doc:
        DOC_NO = audit_doc['rcept_no']
        print(f"📌 감사보고서 발견: {audit_doc['report_nm']}, 접수일: {audit_doc['rcept_dt']}, 문서번호: {DOC_NO}")

        # ✅ 3. 문서 다운로드
        DOC_URL = f'https://opendart.fss.or.kr/api/document.xml?crtfc_key={API_KEY}&rcept_no={DOC_NO}'
        doc_response = requests.get(DOC_URL)

        if doc_response.status_code == 200:
            zip_file_path = f'audit_report_{DOC_NO}.zip'
            with open(zip_file_path, 'wb') as file:
                file.write(doc_response.content)
            print(f'✅ XML 압축 파일 다운로드 완료: {zip_file_path}')

            # ✅ 4. ZIP 파일 해제
            extracted_folder = f'extracted_report_{DOC_NO}/'
            os.makedirs(extracted_folder, exist_ok=True)

            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(extracted_folder)
            print(f"✅ 압축 해제 완료: {extracted_folder}")

            # ✅ 5. XML 파싱 및 재무제표 추출
            xml_files = [f for f in os.listdir(extracted_folder) if f.endswith('.xml')]
            if xml_files:
                xml_file_path = os.path.join(extracted_folder, xml_files[0])

                tree = ET.parse(xml_file_path)
                root = tree.getroot()

                # 재무제표 트리 구조 추출 함수 (자산총계, 부채총계, 자본총계만 필터링)
                def extract_financial_summary(root):
                    financial_summary = []
                    for tr in root.iter('TR'):
                        td_list = list(tr.iter('TD'))
                        if len(td_list) >= 8:  # 8개 열이 존재해야 제대로 된 정보가 있음
                            항목 = td_list[0].text.strip() if td_list[0].text else None
                            당기금액 = td_list[3].text.strip() if td_list[3].text else None  # 당기 금액
                            전기금액 = td_list[6].text.strip() if len(td_list) > 6 and td_list[6].text else None  # 전기 금액

                            # 자산총계, 부채총계, 자본총계 항목만 필터링
                            if 항목 in ["자산총계", "부채총계", "자본총계"]:
                                # 전기금액 처리: 공백 또는 '　' 등의 특수 문자가 있을 수 있으므로 이를 'None'으로 처리
                                if 전기금액 in [None, '　', '']:
                                    전기금액 = None
                                financial_summary.append([항목, 당기금액, 전기금액])

                    return financial_summary

                # 재무제표 요약 추출
                financial_summary = extract_financial_summary(root)

                # ✅ 6. 데이터프레임으로 변환
                df = pd.DataFrame(financial_summary, columns=['항목', '당기 금액', '전기 금액'])

                # ✅ 7. 엑셀 저장
                output_excel = f'financial_summary_{DOC_NO}.xlsx'
                df.to_excel(output_excel, index=False)
                print(f"✅ 재무제표 엑셀 저장 완료: {output_excel}")

            else:
                print("❌ XML 파일이 없습니다.")
        else:
            print(f'❌ XML 다운로드 실패: {doc_response.status_code}')


📌 감사보고서 발견: 감사보고서 (2023.12), 접수일: 20240412, 문서번호: 20240412001149
✅ XML 압축 파일 다운로드 완료: audit_report_20240412001149.zip
✅ 압축 해제 완료: extracted_report_20240412001149/
✅ 재무제표 엑셀 저장 완료: financial_summary_20240412001149.xlsx


In [44]:
import requests
import xml.etree.ElementTree as ET
import os
import zipfile
import pandas as pd

# ✅ 1. DART API 설정
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'  # 실제 키로 변경 필요
CORP_CODE = '01359295'  # 기업 코드 (예제: 삼성전자)
START_DATE = '20240101'
END_DATE = '20241231'

# ✅ 2. 공시 목록 조회 (감사보고서 필터링)
URL = f'https://opendart.fss.or.kr/api/list.json?crtfc_key={API_KEY}&corp_code={CORP_CODE}&bgn_de={START_DATE}&end_de={END_DATE}&page_no=1&page_count=20'
response = requests.get(URL)
data = response.json()

if data['status'] == '000':
    doc_list = data.get('list', [])

    # 🔍 "연결감사보고서"가 아닌 "감사보고서"만 찾기
    audit_doc = next((doc for doc in doc_list if '감사보고서' in doc['report_nm'] and '연결' not in doc['report_nm']), None)

    if audit_doc:
        DOC_NO = audit_doc['rcept_no']
        print(f"📌 감사보고서 발견: {audit_doc['report_nm']}, 접수일: {audit_doc['rcept_dt']}, 문서번호: {DOC_NO}")

        # ✅ 3. 문서 다운로드
        DOC_URL = f'https://opendart.fss.or.kr/api/document.xml?crtfc_key={API_KEY}&rcept_no={DOC_NO}'
        doc_response = requests.get(DOC_URL)

        if doc_response.status_code == 200:
            zip_file_path = f'audit_report_{DOC_NO}.zip'
            with open(zip_file_path, 'wb') as file:
                file.write(doc_response.content)
            print(f'✅ XML 압축 파일 다운로드 완료: {zip_file_path}')

            # ✅ 4. ZIP 파일 해제
            extracted_folder = f'extracted_report_{DOC_NO}/'
            os.makedirs(extracted_folder, exist_ok=True)

            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(extracted_folder)
            print(f"✅ 압축 해제 완료: {extracted_folder}")

            # ✅ 5. XML 파싱 및 재무제표 추출
            xml_files = [f for f in os.listdir(extracted_folder) if f.endswith('.xml')]
            if xml_files:
                xml_file_path = os.path.join(extracted_folder, xml_files[0])

                tree = ET.parse(xml_file_path)
                root = tree.getroot()

                # 재무제표 트리 구조 추출 함수
                def extract_financial_tree(root):
                    financial_tree = []
                    current_parent = None  # 현재 부모 항목을 추적
                    for tr in root.iter('TR'):
                        td_list = list(tr.iter('TD'))
                        if len(td_list) >= 4:
                            항목 = td_list[0].text.strip() if td_list[0].text else None
                            금액 = td_list[3].text.strip() if td_list[3].text else None  # 당기 금액

                            if 항목 and 금액 and '주석' not in 항목:
                                # 자산 항목을 추출하고 트리 구조에 추가
                                if "자산" in 항목:
                                    current_parent = 항목
                                    financial_tree.append([current_parent, 항목, 금액])

                                # 유동자산 항목을 추출하고 자산 하위에 추가
                                elif "유동자산" in 항목 and current_parent == "자산":
                                    current_parent = 항목
                                    financial_tree.append([current_parent, 항목, 금액])

                                # 현금및현금성자산 항목을 추출하고 유동자산 하위에 추가
                                elif "현금및현금성자산" in 항목 and current_parent == "유동자산":
                                    financial_tree.append([current_parent, 항목, 금액])

                    return financial_tree

                # 재무제표 트리 추출
                financial_tree = extract_financial_tree(root)

                # ✅ 6. 데이터프레임으로 변환
                df = pd.DataFrame(financial_tree, columns=['상위 항목', '하위 항목', '금액'])

                # ✅ 7. 엑셀 저장
                output_excel = f'financial_statements_{DOC_NO}.xlsx'
                df.to_excel(output_excel, index=False)
                print(f"✅ 재무제표 엑셀 저장 완료: {output_excel}")

            else:
                print("❌ XML 파일이 없습니다.")
        else:
            print(f'❌ XML 다운로드 실패: {doc_response.status_code}')
    else:
        print("❌ 2024년에 제출된 개별 감사보고서가 없습니다.")
else:
    print(f"❌ 공시 목록 조회 실패: {data['message']}")


📌 감사보고서 발견: 감사보고서 (2023.12), 접수일: 20240412, 문서번호: 20240412001149
✅ XML 압축 파일 다운로드 완료: audit_report_20240412001149.zip
✅ 압축 해제 완료: extracted_report_20240412001149/
✅ 재무제표 엑셀 저장 완료: financial_statements_20240412001149.xlsx


In [46]:
import requests
import xml.etree.ElementTree as ET
import os
import zipfile
import pandas as pd

# ✅ 1. DART API 설정
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'  # 실제 키로 변경 필요
CORP_CODE = '01359295'  # 기업 코드 (예제: 삼성전자)
START_DATE = '20240101'
END_DATE = '20241231'

# ✅ 2. 공시 목록 조회 (감사보고서 필터링)
URL = f'https://opendart.fss.or.kr/api/list.json?crtfc_key={API_KEY}&corp_code={CORP_CODE}&bgn_de={START_DATE}&end_de={END_DATE}&page_no=1&page_count=20'
response = requests.get(URL)
data = response.json()

if data['status'] == '000':
    doc_list = data.get('list', [])

    # 🔍 "연결감사보고서"가 아닌 "감사보고서"만 찾기
    audit_doc = next((doc for doc in doc_list if '감사보고서' in doc['report_nm'] and '연결' not in doc['report_nm']), None)

    if audit_doc:
        DOC_NO = audit_doc['rcept_no']
        print(f"📌 감사보고서 발견: {audit_doc['report_nm']}, 접수일: {audit_doc['rcept_dt']}, 문서번호: {DOC_NO}")

        # ✅ 3. 문서 다운로드
        DOC_URL = f'https://opendart.fss.or.kr/api/document.xml?crtfc_key={API_KEY}&rcept_no={DOC_NO}'
        doc_response = requests.get(DOC_URL)

        if doc_response.status_code == 200:
            zip_file_path = f'audit_report_{DOC_NO}.zip'
            with open(zip_file_path, 'wb') as file:
                file.write(doc_response.content)
            print(f'✅ XML 압축 파일 다운로드 완료: {zip_file_path}')

            # ✅ 4. ZIP 파일 해제
            extracted_folder = f'extracted_report_{DOC_NO}/'
            os.makedirs(extracted_folder, exist_ok=True)

            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(extracted_folder)
            print(f"✅ 압축 해제 완료: {extracted_folder}")

            # ✅ 5. XML 파싱 및 재무제표 추출
            xml_files = [f for f in os.listdir(extracted_folder) if f.endswith('.xml')]
            if xml_files:
                xml_file_path = os.path.join(extracted_folder, xml_files[0])

                tree = ET.parse(xml_file_path)
                root = tree.getroot()

                # 재무제표 트리 구조 추출 함수
                def extract_financial_tree(root):
                    financial_tree = []
                    current_parent = None  # 현재 부모 항목을 추적
                    for tr in root.iter('TR'):
                        td_list = list(tr.iter('TD'))
                        if len(td_list) >= 4:
                            항목 = td_list[0].text.strip() if td_list[0].text else None
                            당기금액 = td_list[3].text.strip() if td_list[3].text else None  # 당기 금액

                            # 전기금액이 있을 때만 처리
                            전기금액 = td_list[6].text.strip() if len(td_list) > 6 and td_list[6].text else None  # 전기 금액

                            if 항목 and (당기금액 or 전기금액) and '주석' not in 항목:
                                # 자산 항목을 추출하고 트리 구조에 추가
                                if "자산" in 항목:
                                    current_parent = 항목
                                    financial_tree.append([current_parent, 항목, 당기금액, 전기금액])

                                # 유동자산 항목을 추출하고 자산 하위에 추가
                                elif "유동자산" in 항목 and current_parent == "자산":
                                    current_parent = 항목
                                    financial_tree.append([current_parent, 항목, 당기금액, 전기금액])

                                # 유동부채 항목을 추출하고 부채 하위에 추가
                                elif "유동부채" in 항목 and "부채" in current_parent:
                                    current_parent = 항목
                                    financial_tree.append([current_parent, 항목, 당기금액, 전기금액])

                                # 부채 항목을 추출하고 트리 구조에 추가
                                elif "부채" in 항목:
                                    current_parent = 항목
                                    financial_tree.append([current_parent, 항목, 당기금액, 전기금액])

                                # 자본 항목을 추출하고 트리 구조에 추가
                                elif "자본" in 항목:
                                    current_parent = 항목
                                    financial_tree.append([current_parent, 항목, 당기금액, 전기금액])

                    return financial_tree

                # 재무제표 트리 추출
                financial_tree = extract_financial_tree(root)

                # ✅ 6. 데이터프레임으로 변환
                df = pd.DataFrame(financial_tree, columns=['상위 항목', '하위 항목', '당기 금액', '전기 금액'])

                # ✅ 7. 엑셀 저장
                output_excel = f'financial_statements_{DOC_NO}.xlsx'
                df.to_excel(output_excel, index=False)
                print(f"✅ 재무제표 엑셀 저장 완료: {output_excel}")

            else:
                print("❌ XML 파일이 없습니다.")
        else:
            print(f'❌ XML 다운로드 실패: {doc_response.status_code}')
    else:
        print("❌ 2024년에 제출된 개별 감사보고서가 없습니다.")
else:
    print(f"❌ 공시 목록 조회 실패: {data['message']}")


📌 감사보고서 발견: 감사보고서 (2023.12), 접수일: 20240412, 문서번호: 20240412001149
✅ XML 압축 파일 다운로드 완료: audit_report_20240412001149.zip
✅ 압축 해제 완료: extracted_report_20240412001149/
✅ 재무제표 엑셀 저장 완료: financial_statements_20240412001149.xlsx


In [54]:
import requests
import xml.etree.ElementTree as ET
import os
import zipfile
import pandas as pd

# ✅ 1. DART API 설정
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'  # 실제 키로 변경 필요
CORP_CODE = '01728547'  # 기업 코드 (예제: 삼성전자)
START_DATE = '20240101'
END_DATE = '20241231'

# ✅ 2. 공시 목록 조회 (감사보고서 필터링)
URL = f'https://opendart.fss.or.kr/api/list.json?crtfc_key={API_KEY}&corp_code={CORP_CODE}&bgn_de={START_DATE}&end_de={END_DATE}&page_no=1&page_count=20'
response = requests.get(URL)
data = response.json()

if data['status'] == '000':
    doc_list = data.get('list', [])

    # 🔍 "연결감사보고서"가 아닌 "감사보고서"만 찾기
    audit_doc = next((doc for doc in doc_list if '감사보고서' in doc['report_nm'] and '연결' not in doc['report_nm']), None)

    if audit_doc:
        DOC_NO = audit_doc['rcept_no']
        print(f"📌 감사보고서 발견: {audit_doc['report_nm']}, 접수일: {audit_doc['rcept_dt']}, 문서번호: {DOC_NO}")

        # ✅ 3. 문서 다운로드
        DOC_URL = f'https://opendart.fss.or.kr/api/document.xml?crtfc_key={API_KEY}&rcept_no={DOC_NO}'
        doc_response = requests.get(DOC_URL)

        if doc_response.status_code == 200:
            zip_file_path = f'audit_report_{DOC_NO}.zip'
            with open(zip_file_path, 'wb') as file:
                file.write(doc_response.content)
            print(f'✅ XML 압축 파일 다운로드 완료: {zip_file_path}')

            # ✅ 4. ZIP 파일 해제
            extracted_folder = f'extracted_report_{DOC_NO}/'
            os.makedirs(extracted_folder, exist_ok=True)

            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(extracted_folder)
            print(f"✅ 압축 해제 완료: {extracted_folder}")

            # ✅ 5. XML 파싱 및 재무제표 추출
            xml_files = [f for f in os.listdir(extracted_folder) if f.endswith('.xml')]
            if xml_files:
                xml_file_path = os.path.join(extracted_folder, xml_files[0])

                tree = ET.parse(xml_file_path)
                root = tree.getroot()

                # 재무제표 트리 구조 추출 함수
                def extract_financial_summary(root):
                    financial_summary = []
                    seen_항목 = set()  # 중복된 항목을 체크할 set
                    current_parent = None  # 현재 상위 항목을 추적
                    for tr in root.iter('TR'):
                        td_list = list(tr.iter('TD'))
                        if len(td_list) >= 4:
                            항목 = td_list[0].text.strip() if td_list[0].text else None
                            당기금액 = td_list[3].text.strip() if td_list[3].text else None  # 당기 금액
                            전기금액 = td_list[6].text.strip() if len(td_list) > 6 and td_list[6].text else None  # 전기 금액

                            # 중복 체크 후 추가
                            if 항목 and 항목 not in seen_항목:
                                seen_항목.add(항목)  # 항목을 set에 추가하여 중복을 피함
                                if current_parent:
                                    # 상위 항목이 있을 경우 하위 항목으로 추가 (순서를 바꿈)
                                    financial_summary.append([항목, current_parent, 당기금액, 전기금액])  # 상위/하위 항목 교환
                                else:
                                    # 최상위 항목인 경우
                                    financial_summary.append([항목, None, 당기금액, 전기금액])

                            # 상위 항목을 추적
                            if "자산" in 항목:
                                current_parent = 항목  # 자산이 상위 항목이면 그 다음부터 하위 항목으로 처리

                    return financial_summary

                # 재무제표 요약 추출
                financial_summary = extract_financial_summary(root)

                # ✅ 6. 데이터프레임으로 변환
                df = pd.DataFrame(financial_summary, columns=['하위 항목', '상위 항목', '당기 금액', '전기 금액'])

                # ✅ 7. 엑셀 저장
                output_excel = f'financial_summary_{DOC_NO}.xlsx'
                df.to_excel(output_excel, index=False)
                print(f"✅ 재무제표 엑셀 저장 완료: {output_excel}")

            else:
                print("❌ XML 파일이 없습니다.")
        else:
            print(f'❌ XML 다운로드 실패: {doc_response.status_code}')
    else:
        print("❌ 2024년에 제출된 개별 감사보고서가 없습니다.")
else:
    print(f"❌ 공시 목록 조회 실패: {data['message']}")


📌 감사보고서 발견: 감사보고서 (2023.12), 접수일: 20240408, 문서번호: 20240408003387
✅ XML 압축 파일 다운로드 완료: audit_report_20240408003387.zip
✅ 압축 해제 완료: extracted_report_20240408003387/


ParseError: mismatched tag: line 2422, column 91 (<string>)

In [62]:
import requests
import os
import zipfile
import pandas as pd
from lxml import etree

# ✅ 1. DART API 설정
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'  # 실제 키로 변경 필요
CORP_CODE = '01728547'  # 기업 코드 (예제: 삼성전자)
START_DATE = '20240101'
END_DATE = '20241231'

# ✅ 2. 공시 목록 조회 (감사보고서 필터링)
URL = f'https://opendart.fss.or.kr/api/list.json?crtfc_key={API_KEY}&corp_code={CORP_CODE}&bgn_de={START_DATE}&end_de={END_DATE}&page_no=1&page_count=20'
response = requests.get(URL)
data = response.json()

if data['status'] == '000':
    doc_list = data.get('list', [])

    # 🔍 "연결감사보고서"가 아닌 "감사보고서"만 찾기
    audit_doc = next((doc for doc in doc_list if '감사보고서' in doc['report_nm'] and '연결' not in doc['report_nm']), None)

    if audit_doc:
        DOC_NO = audit_doc['rcept_no']
        print(f"📌 감사보고서 발견: {audit_doc['report_nm']}, 접수일: {audit_doc['rcept_dt']}, 문서번호: {DOC_NO}")

        # ✅ 3. 문서 다운로드
        DOC_URL = f'https://opendart.fss.or.kr/api/document.xml?crtfc_key={API_KEY}&rcept_no={DOC_NO}'
        doc_response = requests.get(DOC_URL)

        if doc_response.status_code == 200:
            zip_file_path = f'audit_report_{DOC_NO}.zip'
            with open(zip_file_path, 'wb') as file:
                file.write(doc_response.content)
            print(f'✅ XML 압축 파일 다운로드 완료: {zip_file_path}')

            # ✅ 4. ZIP 파일 해제
            extracted_folder = f'extracted_report_{DOC_NO}/'
            os.makedirs(extracted_folder, exist_ok=True)

            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(extracted_folder)
            print(f"✅ 압축 해제 완료: {extracted_folder}")

            # ✅ 5. XML 파싱 및 재무제표 추출
            xml_files = [f for f in os.listdir(extracted_folder) if f.endswith('.xml')]
            if xml_files:
                xml_file_path = os.path.join(extracted_folder, xml_files[0])

                # lxml을 사용하여 XML 파싱
                def parse_xml_with_lxml(xml_file_path):
                    try:
                        tree = etree.parse(xml_file_path)
                        root = tree.getroot()
                        return root
                    except etree.XMLSyntaxError as e:
                        print(f"XML 파싱 오류: {e}")
                        return None

                # XML 파일 파싱
                root = parse_xml_with_lxml(xml_file_path)
                if root is not None:
                    # 재무제표 트리 구조 추출 함수
                    def extract_financial_tree(root):
                        financial_tree = []
                        seen = set()  # 중복된 항목을 체크할 set
                        current_parent = None  # 현재 부모 항목을 추적
                        for tr in root.iter('TR'):
                            td_list = list(tr.iter('TD'))
                            if len(td_list) >= 4:
                                항목 = td_list[0].text.strip() if td_list[0].text else None
                                금액 = td_list[3].text.strip() if td_list[3].text else None  # 당기 금액
                                전기금액 = td_list[6].text.strip() if len(td_list) > 6 else None  # 전기 금액

                                if 항목 and (금액 or 전기금액) and '주석' not in 항목 and 항목 not in seen:
                                    # 상위 항목과 하위 항목 구분하여 추출
                                    if "자산" in 항목:
                                        current_parent = 항목
                                        financial_tree.append([항목, 항목, 금액])
                                        seen.add(항목)
                                    elif "유동자산" in 항목 and "자산" in current_parent:
                                        current_parent = 항목
                                        financial_tree.append([current_parent, 항목, 금액])
                                        seen.add(항목)
                                    elif "유동부채" in 항목 and "부채" in current_parent:
                                        current_parent = 항목
                                        financial_tree.append([current_parent, 항목, 금액])
                                        seen.add(항목)
                                    elif "자본" in 항목:
                                        current_parent = 항목
                                        financial_tree.append([항목, 항목, 금액])
                                        seen.add(항목)
                                    elif 항목 != current_parent:  # 중복 항목을 제외한 하위 항목 처리
                                        financial_tree.append([current_parent, 항목, 금액])
                                        seen.add(항목)

                        return financial_tree

                    # 재무제표 트리 추출
                    financial_tree = extract_financial_tree(root)

                    # ✅ 6. 데이터프레임으로 변환
                    df = pd.DataFrame(financial_tree, columns=['상위 항목', '하위 항목', '금액'])

                    # ✅ 7. 엑셀 저장
                    output_excel = f'financial_statements_{DOC_NO}.xlsx'
                    df.to_excel(output_excel, index=False)
                    print(f"✅ 재무제표 엑셀 저장 완료: {output_excel}")

                else:
                    print("❌ XML 파싱 실패")
            else:
                print("❌ XML 파일이 없습니다.")
        else:
            print(f'❌ XML 다운로드 실패: {doc_response.status_code}')
    else:
        print("❌ 2024년에 제출된 개별 감사보고서가 없습니다.")
else:
    print(f"❌ 공시 목록 조회 실패: {data['message']}")


📌 감사보고서 발견: 감사보고서 (2023.12), 접수일: 20240408, 문서번호: 20240408003387
✅ XML 압축 파일 다운로드 완료: audit_report_20240408003387.zip
✅ 압축 해제 완료: extracted_report_20240408003387/
XML 파싱 오류: Opening and ending tag mismatch: 당기 line 2422 and P, line 2422, column 94 (20240408003387_00760.xml, line 2422)
❌ XML 파싱 실패


##--------------------------------------------------